In [1]:
import configparser
import dataclasses
import os
import re
import shutil
import statistics
from collections import Counter
from itertools import chain
from pathlib import Path
from time import perf_counter

import pandas as pd
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from IPython.display import display
import logging

import utils
from fns import *

## START LOGGING 

In [2]:
Path('logs').mkdir(parents=True, exist_ok=True)
log = utils.make_logger("logs/rp_log")
log.info("\nSTART")
log.info("\nCHECKING CONFIGURATION FILE CONTENTS...")

## GET PARAMS FROM CONFIG FILE

In [3]:
# checking config file
# if any error occurs then program terminates; then check the config file
config = configparser.ConfigParser()
config.read_file(open('config.ini'))

abi_sequence_folder = checking_dirs(config['Paths']['abi_sequence_folder'], log, log_msg=True, create_dir=False)
vh_template_sequence_folder = checking_dirs(config['Paths']['vh_template_sequence_folder'], log, log_msg=True, create_dir=False)
vl_template_sequence_folder = checking_dirs(config['Paths']['vl_template_sequence_folder'], log, log_msg=True, create_dir=False)
results_dir = checking_dirs(config['Paths']['results_dir'], log, log_msg=True, create_dir=True)
h3_nt_data_sheet_filepath = check_files(config['Files']['h3_nt_data_sheet_filepath'], log)
df = check_tsv_file(h3_nt_data_sheet_filepath, log)
excel_path_file_name = create_results_excel_file_path(config['Paths']['results_dir'], config['Files']['output_excel_file_name'])

# patterns
pat_vh, pat_vl = get_patterns(config)
# patterns to remove from abi names
patrm_vh_abi, patrm_vl_abi = get_patterns_to_rm(config)
# patterns to remove from genbank names
patrm_vh_gb, patrm_vl_gb = get_patterns_to_rm_from_genbank(config)

# get alignment parameters
par_match, par_missmatch, par_open, par_extend, par_filter_thresh = get_alignment_params(config)

# get alignment start and end regions
vh_seq_start_a, vh_seq_end_a, vl_seq_start_a, vl_seq_end_a = get_alignment_seq_start_and_end(config)

# creating dirs for copying matched abi files
res_dir_vh = checking_dirs(f"{results_dir}/{pat_vh}", log, log_msg=True, create_dir=True)
res_dir_vl = checking_dirs(f"{results_dir}/{pat_vl}", log, log_msg=True, create_dir=True)


## GET ABI FILES

In [4]:
log.info("\nCHECKING THE AB1 FILES...")
vh_abi_dict = {i.name.replace('.abi', '').replace(patrm_vh_abi, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vh}*.abi")])}
vl_abi_dict = {i.name.replace('.abi', '').replace(patrm_vl_abi, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vl}*.abi")])}

if len(vh_abi_dict) == len(vl_abi_dict):
    log.info(f"[+] INFO: There are {len(vh_abi_dict)} abi files in the abi sequence folder")
else:
    log.warning(f"[+] WARN: There is a differrence in number of abi files in vh: {len(vh_abi_dict)} and vl {len(vl_abi_dict)}")

In [5]:
sample_ids = checking_ab1_files(log, vh_abi_dict, vl_abi_dict)

In [6]:
log.info("\nCHECKING THE GENBANK FILES...")
vh_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vh_template_sequence_folder).glob('*.gb'))}
vl_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vl_template_sequence_folder).glob('*.gb'))}

if len(vh_template_gb_dict) == len(vl_template_gb_dict):
    log.info(f"[+] INFO: There are {len(vh_template_gb_dict)} genbank files in each dir vh and vl")
else:
    log.warning(f"[+] WARN: There is a differrence in number of genbank files in vh: {len(vh_template_gb_dict)} and vl {len(vl_template_gb_dict)}")

## GET PROBE SEQS INTO A DICTIOANRY

In [7]:
# dictionary containing probe seq and name
h3_dict = df.set_index('name', drop=True).to_dict().get('h3_nt')

## FINDING THE PROBES THAT MATCH VH AND CORRESPONDING VL FILES

In [8]:
# this loop  iternate through all the sample ids and find_match_on_all_h3probes function search for the probe on the normal seq and the rev complemented seq
# if there is a match which will be returened as a list

log.info(f"\nITERATING THROUGH EACH SAMPLE ID")
result_vh, result_vl = [], []
for sample in sample_ids:
    # print(f">>{sample}")
    _vhabi, _vlabi = get_abi_file_path(key=sample, vh_abi_dict=vh_abi_dict, vl_abi_dict=vl_abi_dict)
    vh_d = get_seqobj_from_abi(_vhabi)  # returns a seq record obj of VH
    vl_d = get_seqobj_from_abi(_vlabi)  # returns a seq record obj of VL
    
    # matching each probe on  VH and VL - normal and revcomp sequence
    vh_prob_search = find_match_on_all_h3probes_v2(log, h3_dict, vh_d, sample, vh_abi_dict, vl_abi_dict)
    vl_prob_search = find_match_on_all_h3probes_v2(log, h3_dict, vl_d, sample, vh_abi_dict, vl_abi_dict)
    
    if len(vh_prob_search) >=1:
        result_vh.append(vh_prob_search)
    if len(vl_prob_search) >=1:
        result_vl.append(vl_prob_search)
log.info(f"\nFINISH ITERATING THROUGH EACH SAMPLE ID")
# if vh and vl ids match then copy the files to a new place
colnames=["Match","h3_name","sample_id","vh_abi_fp","vl_abi_fp","probe_seq", "seq_record", "abi_seq", "trimmed_seq", "trimmed_seq_qual"]
df_vh = pd.DataFrame(chain.from_iterable(result_vh))
df_vl = pd.DataFrame(chain.from_iterable(result_vl))
df_vh.columns, df_vl.columns = colnames, colnames

In [9]:
df_vh.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,seq_record,abi_seq,trimmed_seq,trimmed_seq_qual
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ...","(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(G, G, G, C, C, T, G, C, C, C, C, A, G, A, A, ...","[54, 7, 6, 12, 16, 16, 37, 35, 41, 39, 19, 24,..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ...","(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(T, A, C, T, C, A, C, C, T, G, A, G, C, T, C, ...","[58, 47, 37, 58, 31, 24, 54, 58, 58, 58, 45, 5..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ...","(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, T, G, G, C, C, C, C, A, G, A, C, G, T, C, ...","[50, 56, 59, 43, 39, 56, 56, 43, 36, 34, 49, 3..."


In [10]:
df_vl.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,seq_record,abi_seq,trimmed_seq,trimmed_seq_qual
0,Forward_Strand_Match,TMH577-hF-014-G03,TMH577-hIgG1-013-A3_A03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCAAGAGGAATTCGCCGTATTACTATGGTTCGGGGAGCTGGGGGGT...,"(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ...","(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 22, 18, 52, 52, 58, 41, 14, 28, 54, 58, 5..."
1,Forward_Strand_Match,TMH577-hF-012-F03,TMH577-hIgG1-013-D10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGGGGCGGATACAGCTATGGCATTGACTAC,"(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ...","(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ...","(T, G, A, C, A, G, G, G, A, G, G, G, C, A, G, ...","[53, 47, 54, 14, 16, 35, 29, 20, 13, 20, 58, 2..."
2,Forward_Strand_Match,TMH577-hF-012-C01,TMH577-hIgG1-013-E11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGCGGGTGGGAGCCGCGGCCGATGCTTTTGATATC,"(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ...","(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ...","(A, C, A, G, G, G, A, G, G, G, C, A, G, A, G, ...","[54, 13, 11, 41, 17, 17, 19, 26, 58, 23, 18, 4..."
3,Forward_Strand_Match,TMH577-hF-016-C12,TMH577-hIgG1-013-F3_F03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGAGTGACTACGGCCGACTAC,"(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ...","(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ...","(G, G, C, A, G, A, G, G, T, C, C, T, T, G, C, ...","[58, 25, 25, 52, 49, 58, 45, 30, 35, 58, 58, 5..."
4,Forward_Strand_Match,TMH577-hF-017-C01,TMH577-hIgG1-013-G3_G03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAATCTGTCTGCGTGGGAAGTACCC,"(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ...","(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ...","(G, G, G, A, G, G, G, C, A, G, A, G, G, T, C, ...","[54, 29, 15, 12, 25, 58, 22, 15, 48, 52, 54, 4..."


In [11]:
# if the sahpe of df_vh and df_vl are same and the h3_name in both dataframes are same then copy the matched data to a new dir
if(df_vl.shape == df_vh.shape) and (df_vl.h3_name == df_vh.h3_name).all():
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in the dataframe between vh and vl")
    log.info(f"[+] INFO: Of which {df_vh[['vh_abi_fp', 'vl_abi_fp']].drop_duplicates().shape[0]} samples has to be moved to a new dir")
else:
    log.info(f"[+] INFO: There are {df_vh.shape[0]} matches in df_vh")
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in df_vl")

## COPY PROBE MATCHED ABI FILES TO A NEW LOC

In [12]:
log.info(f"\nCOPY MATCHED ABI FILES INTO NEW LOCATION")
res_df_copy = copy_mtched_abi_files_to_resdir(log, res_dir_vh, res_dir_vl, df_vh, log_msg=True)
log.info(f"\nFINISH COPYING FILES")

In [13]:
# this dataframe contain the new and old location of abi files 
res_df_copy.head()

,h3_name,sample_id,abi_out_loc,abi_initial_filepath
0,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,TMH577-hF-012-E03,TMH577-hIgG1-013-H3_H03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,TMH577-hF-012-E03,TMH577-hIgG1-013-H7_H07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,TMH577-hF-012-E03,TMH577-hIgG1-014-A5_A05,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,TMH577-hF-012-E03,TMH577-hIgG1-014-C8_C08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


## Note
- Save a copy of df_vl, df_vh, res_df_copy

### Sanity check

In [14]:
df_vh.shape, df_vl.shape, res_df_copy.shape

((183, 10), (23, 10), (366, 4))

In [15]:
res_df_copy.h3_name.value_counts()

TMH577-hF-014-B12    22
TMH577-hF-015-C08    22
TMH577-hF-015-D08    22
TMH577-hF-017-G04    22
TMH577-hF-012-E03    14
                     ..
TMH577-hF-016-D06     2
TMH577-hF-012-C01     2
TMH577-hF-016-C03     2
TMH577-hF-004-G09     2
TMH577-hF-017-D03     2
Name: h3_name, Length: 67, dtype: int64

In [16]:
sample_ids[0:5]

['TMH577-hIgG1-013-A10',
 'TMH577-hIgG1-013-A11',
 'TMH577-hIgG1-013-A12',
 'TMH577-hIgG1-013-A1_A01',
 'TMH577-hIgG1-013-A2_A02']

## PAIRWISE ALIGNMENT OF GENBANK FILES WITH MATCHED ABI FILES TO GET THE MATCHING SCORE 
- Iterate through the VH (and VL) genbank file names [the file names are in a dict vh_template_gb_dict]
    - if the gb file name matches with h3_name column in df_vh or df_vl 
      - then save that into a list gb_match_with_dfvx_lst
          - GB_Matched|gb_id|h3_name|sample_id|gb_fp|vh_abi_fp|vl_abi_fp
- gb file name could match an h3_name; also this h3_name might be associated with different sample ids
```
[+] GENBANK_MATCHING:|genbank_id|h3_name|sample_id
[+] GENBANK_MATCHING:|VH-TMH577-hF-005-G08|TMH577-hF-005-G08|TMH577-hIgG1-014-A8_A08
[+] GENBANK_MATCHING:|VH-TMH577-hF-005-G08|TMH577-hF-005-G08|TMH577-hIgG1-014-C4_C04
```

In [17]:
log.info(f"\nFIND THE GB FILENASMES WITH H3 NAMES")
M_gb_abi_vh = find_gb_match_on_all_h3probes_v2(log, vh_template_gb_dict, df_vh, pattern=patrm_vh_gb, log_msg=True)
M_gb_abi_vl = find_gb_match_on_all_h3probes_v2(log, vl_template_gb_dict, df_vl, pattern=patrm_vl_gb, log_msg=True)

In [19]:
# M_gb_abi_vh = find_gb_match_on_all_h3probes_v2(log, vh_template_gb_dict, df_vh, pattern=patrm_vh_gb, log_msg=False)

In [20]:
M_gb_abi_vh

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...,...,...,...
178,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
179,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
180,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
181,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [21]:
M_gb_abi_vl

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VL-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VL-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VL-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VL-TMH577-hF-012-C01,TMH577-hF-012-C01,TMH577-hIgG1-013-E11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VL-TMH577-hF-012-F03,TMH577-hF-012-F03,TMH577-hIgG1-013-D10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
5,GB_Matched,VL-TMH577-hF-014-B12,TMH577-hF-014-B12,TMH577-hIgG1-014-C7_C07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
6,GB_Matched,VL-TMH577-hF-014-B12,TMH577-hF-014-B12,TMH577-hIgG1-014-F7_F07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
7,GB_Matched,VL-TMH577-hF-014-D12,TMH577-hF-014-D12,TMH577-hIgG1-014-D8_D08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
8,GB_Matched,VL-TMH577-hF-014-G03,TMH577-hF-014-G03,TMH577-hIgG1-013-A3_A03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
9,GB_Matched,VL-TMH577-hF-015-C08,TMH577-hF-015-C08,TMH577-hIgG1-014-C7_C07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [22]:
all_abi_probe_match  = pd.concat([df_vh, df_vl]).reset_index(drop=True)
all_gb_match = pd.concat([M_gb_abi_vh, M_gb_abi_vl]).reset_index(drop=True)

In [23]:
all_gb_match.shape

(206, 7)

In [24]:
all_gb_match[['gb_id', 'h3_name', 'sample_id']].drop_duplicates()

,gb_id,h3_name,sample_id
0,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04
1,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08
2,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04
3,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12
4,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09
...,...,...,...
201,VL-TMH577-hF-017-B04,TMH577-hF-017-B04,TMH577-hIgG1-014-A12
202,VL-TMH577-hF-017-C01,TMH577-hF-017-C01,TMH577-hIgG1-013-G3_G03
203,VL-TMH577-hF-017-C01,TMH577-hF-017-C01,TMH577-hIgG1-014-B10
204,VL-TMH577-hF-017-G04,TMH577-hF-017-G04,TMH577-hIgG1-014-C7_C07


#### Sanity check

In [25]:
len(vh_abi_dict),len(vl_abi_dict), len(sample_ids)

(186, 186, 186)

In [26]:
 df_vh.shape, df_vl.shape

((183, 10), (23, 10))

In [27]:
len(vh_template_gb_dict), len(vl_template_gb_dict)

(1917, 1935)

In [28]:
all_gb_match.shape

(206, 7)

## MATCH BETWEEN VH AND GB FILES
- if the score is > threshold save it to a dataframe

In [30]:
def run_alignment_and_filtering(M_gb_abi_vx: pd.DataFrame, 
                                df_vx:pd.DataFrame, 
                                vx_template_gb_dict: dict, 
                                vx_seq_start_a:str, 
                                vx_seq_end_a:str,
                                par_match:int,
                                par_missmatch:int,
                                par_open:int,
                                par_extend:int,
                                par_filter_thresh:int,
                                log :logging, 
                                log_msg: bool=False):
    """
    Returns a dataframe containing gbid,H3_name,GB_FP,ABI_FP,Score,Quality_score,Low_quality
    when for all the datasets where score is above 700
    """
    filtered_res = []
    log.info(f"[+] FILTERED_MATCH: outer_index|inner_index|GB_ID_MATCHED|H3_name|GB_FP|ABI_VH_FP|Score|Quality_score|Low_quality")if log_msg else None
    for e,i in enumerate(range(M_gb_abi_vx.shape[0])[:]):
        gbid = M_gb_abi_vx.iloc[i].gb_id
        gbfp = M_gb_abi_vx.iloc[i].gb_fp
        _dfvx_ss = df_vx[df_vx.h3_name == M_gb_abi_vx.iloc[i].h3_name]   
        for e2, abx in enumerate(range(_dfvx_ss.shape[0])):
            # match between gb and vx-abi
            try:
                seqA = SeqIO.read(vx_template_gb_dict.get(gbid), 'gb')
                seqB = _dfvx_ss.iloc[abx].abi_seq
                seqBinit = _dfvx_ss.iloc[abx].seq_record.seq
                quality = _dfvx_ss.iloc[abx].trimmed_seq_qual
                for a in align(seqA.seq, seqB, match=par_match, mismatch=par_missmatch, gap_open=par_open, gap_extend=par_extend):
                    score = int(a[2])
                    if score >= int(par_filter_thresh):

                        # getting the aligned seqA and seqB
                        aligned_seq_a = a[0]
                        aligned_seq_a = aligned_seq_a.replace('-', '')
                        aligned_seq_a_start = aligned_seq_a[0:6]
                        aligned_seq_a_length = len(aligned_seq_a) - 6
                        aligned_seq_a_end = aligned_seq_a[aligned_seq_a_length:len(aligned_seq_a)]
                        aligned_seq_b = a[1]
                        # futher filtering of seqB
                        if '-' not in aligned_seq_b:
                            if vh_seq_start_a in aligned_seq_a_start:
                                if vh_seq_end_a in aligned_seq_a_end: 
                                    aligned_seq = a[0]
                                    aligned_seq = aligned_seq.replace('-', '')
                                    aligned_seq = Seq(aligned_seq)
                                    aligned_seq_rev = aligned_seq.reverse_complement()
                                    dna_string = str(seqBinit)
                                    aligned_seq_rev = str(aligned_seq_rev)
                                    start = re.search(aligned_seq_rev, dna_string).start()
                                    end = re.search(aligned_seq_rev, dna_string).end()
                                    quality = quality[start:end]
                                    quality_score = statistics.mean(quality)
                                    lowest_quality = min(quality)
                                    # print(f"{e} | {abx} | {gbid} | {_dfvx_ss.iloc[abx].h3_name} | {gbfp} | {_dfvx_ss.iloc[abx].vh_abi_fp} | {score}| {quality_score} | {lowest_quality}")
                                    log.info(f"[+] FILTERED_GOODMATCH: {e}|{abx}|{gbid}|{_dfvx_ss.iloc[abx].sample_id}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|{quality_score}|{lowest_quality}")if log_msg else None
                                    filtered_res.append([gbid, _dfvx_ss.iloc[abx].sample_id, _dfvx_ss.iloc[abx].h3_name, gbfp, _dfvx_ss.iloc[abx].vh_abi_fp, score, quality_score, lowest_quality])
                else:
                    log.info(f"[+] FILTERED_BADMATCH: {e}|{abx}|{gbid}|{_dfvx_ss.iloc[abx].sample_id}|{_dfvx_ss.iloc[abx].h3_name}|{gbfp}|{_dfvx_ss.iloc[abx].vh_abi_fp}|{score}|---|---")if log_msg else None
            except:
                pass
    if not len(filtered_res) == 0:
        dfres = pd.DataFrame(filtered_res)
        dfres.columns = ["gbid", "sample_id", "H3_name", "GB_FP", "ABI_FP", "Score", "Mean_Quality_score", "Low_quality"]
        return dfres
    else:
        return None

In [31]:
vh_gb_abi_match_filtered = run_alignment_and_filtering(M_gb_abi_vh, df_vh, vh_template_gb_dict, vh_seq_start_a, vh_seq_end_a, par_match, par_missmatch, par_open, par_extend, par_filter_thresh, log, log_msg=True)
vl_gb_abi_match_filtered = run_alignment_and_filtering(M_gb_abi_vl, df_vl, vl_template_gb_dict, vl_seq_start_a, vl_seq_end_a, par_match, par_missmatch, par_open, par_extend, par_filter_thresh, log, log_msg=True)

In [37]:
if not isinstance(vh_gb_abi_match_filtered, type(None)):
    vh_gb_abi_match_filtered.to_excel("VH_Match_GB_ABI.xlsx", index=False)
else:
    print("Check the log file")

In [38]:
if not isinstance(vl_gb_abi_match_filtered, type(None)):
    vl_gb_abi_match_filtered.to_excel("VL_Match_GB_ABI.xls", index=False)
else:
    print("Check the log file")

Check the log file
